# draft

최규빈  
2023-09-20

# 4. Pandas: query

In [226]:
df=pd.DataFrame(np.random.normal(size=(20,4)),columns=list('ABCD'),index=pd.date_range('20221226',periods=20)).assign(E=['A']*10+['B']*10)
df

## A. 기본 query

`-` 예시1: `A>0 and B<0`

In [227]:
df.query('A>0 and B<0')

`-` 예시2: `A<B<C`

In [228]:
df.query('A<B<C')

`-` 예시3: `(A+B/2) > 0`

In [229]:
df.query('(A+B)>0')

`-` 예시4: `(A+B/2) > 0` and `E=='A'`

In [230]:
df.query('(A+B)>0 and E=="A"')

In [231]:
df.query("(A+B)>0 and E=='A'")

## B. 외부변수를 이용

`-` 예시

In [205]:
mean = df.A.mean()
mean

In [206]:
df.query('A > @mean')

## C. Index로 query

`-` 예시

In [217]:
df.query('index <= "2022-12-30" or index == "2023-01-10"')

# 5. Pandas: 할당

`-` 자료

In [219]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
df = pd.DataFrame({'att':att,'rep':rep,'mid':mid,'fin':fin})
df

## A. `df.assign()`

`-` 방법1: assign을 이용한 추가

In [220]:
df.assign(total = df.att*0.1 + df.rep*0.2 + df.mid*0.35 + df.fin*0.35) 

> 이 방법은 df를 일시적으로 변화시킴

## B. `df.eval()`

`-` 방법2: eval을 이용한 추가

In [221]:
df.eval('total = att*0.1 + rep*0.2 + mid*0.3 + fin*0.4') 

> 이 방법은 df를 일시적으로 변화시킴

## C. `df[colname] = xxx`

`-` 방법3: `df['total']` 을 이용한 할당

In [183]:
df['total'] = df.att*0.1 + df.rep*0.2 + df.mid*0.3 + df.fin*0.4
df

> 이 방법은 df를 영구적으로 변화시킴

# 6. HW

`-` 문제: 아래의 자료에서 입학년도를 추가하고 싶다면?

In [184]:
np.random.seed(43052)
att = np.random.choice(np.arange(10,21)*5,20)
rep = np.random.choice(np.arange(5,21)*5,20)
mid = np.random.choice(np.arange(0,21)*5,20)
fin = np.random.choice(np.arange(0,21)*5,20)
student_id = [ '2023-12362', '2022-12471', '2023-12333', '2022-12400', '2022-12377',
               '2022-12469', '2023-12314', '2022-12363', '2023-12445', '2023-12336',
               '2023-12426', '2022-12380', '2023-12422', '2022-12488', '2022-12370',
               '2023-12443', '2022-12463', '2023-12491', '2023-12340', '2022-12312' ]
df = pd.DataFrame({'student_id':student_id,'att':att,'rep':rep,'mid':mid,'fin':fin})
df.head()

In [185]:
df.assign(year=[l.split('-')[0] for l in df.student_id]).query('year=="2022"')

# 3. `map(function, list)`

## A. `lambda`

`-` 예시1: $x \to x+2$

In [3]:
f = lambda x: x+2 
f(3)

`-` 예시2: $x,y \to x+y$

In [4]:
f = lambda x,y: x+y 
f(3,5)

`-` 예시3: ‘2023-09’ $\to$ 9

In [16]:
int('2023-09'[-2:])

In [17]:
f = lambda x: int(x[-2:])
f('2023-09')

In [18]:
f('2022-10')

`-` 예시4: ‘2023-09’ $\to$ 2023,9

In [21]:
f = lambda x: (int(x[:4]),int(x[-2:]))
f('2023-09')

`-` 예시5: 문자열이 ‘cat’이면 1 ’dog’ 이면 0

In [23]:
f = lambda x: 1 if x=='cat' else 0 

In [25]:
f('cat'),f('dog')